In [ ]:
#!pip install findspark


In [ ]:
import findspark
findspark.init("/home/bigdata/spark")

In [ ]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('Housing').getOrCreate()

In [ ]:
df = spark.read.csv("/home/bigdata/Desktop/BostonHousing.csv", header=True, inferSchema=True)
# header=True means first row is header


In [ ]:
from pyspark.ml.feature import StringIndexer, VectorAssembler
features = df.columns[:-1]  # Exclude the label column
assembler = VectorAssembler(inputCols=features, outputCol="features")
df = assembler.transform(df)


In [ ]:
train_data, test_data = df.randomSplit([.8,.2],seed=42)


In [ ]:
train_data.columns


['crim',
 'zn',
 'indus',
 'chas',
 'nox',
 'rm',
 'age',
 'dis',
 'rad',
 'tax',
 'ptratio',
 'b',
 'lstat',
 'medv',
 'features']

In [ ]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol='features', labelCol="medv")

In [ ]:
model=lr.fit(train_data)

In [ ]:
predictions = model.transform(test_data)

In [ ]:
evaluation = model.evaluate(test_data)

In [ ]:
evaluation.meanAbsoluteError

3.5080612468325625

In [ ]:
evaluation.rootMeanSquaredError

4.671806485171292

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(labelCol="medv", predictionCol="prediction", metricName="r2")
r2 = evaluator.evaluate(predictions)
print("R² Score:", r2)

R² Score: 0.7931522547835961


In [ ]:
from pyspark.ml.regression import RandomForestRegressor
rf = RandomForestRegressor(labelCol="medv", featuresCol="features")

In [ ]:
model2 = rf.fit(train_data)

In [ ]:
predictions2 = model2.transform(test_data)

In [ ]:
predictions2.show()

+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+--------------------+------------------+
|   crim|  zn|indus|chas|  nox|   rm| age|   dis|rad|tax|ptratio|     b|lstat|medv|            features|        prediction|
+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+--------------------+------------------+
|0.01096|55.0| 2.25|   0|0.389|6.453|31.9|7.3073|  1|300|   15.3|394.72| 8.23|22.0|[0.01096,55.0,2.2...| 23.23856706604841|
|0.01381|80.0| 0.46|   0|0.422|7.875|32.0|5.6484|  4|255|   14.4|394.23| 2.97|50.0|[0.01381,80.0,0.4...| 45.22708600759144|
|0.01439|60.0| 2.93|   0|0.401|6.604|18.8|6.2196|  1|265|   15.6| 376.7| 4.38|29.1|[0.01439,60.0,2.9...|25.966020309941864|
|0.01778|95.0| 1.47|   0|0.403|7.135|13.9|7.6534|  3|402|   17.0| 384.3| 4.45|32.9|[0.01778,95.0,1.4...| 33.63096109110492|
|0.02177|82.5| 2.03|   0|0.415| 7.61|15.7|  6.27|  2|348|   14.7|395.38| 3.11|42.3|[0.02177,82.5,2.0...| 45.25420108695651|
|0.02729

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(labelCol="medv"\
                                , predictionCol="prediction", metricName="rmse")
print ("Root Mean Squared Error (RMSE) on test data = ",evaluator.evaluate(predictions))
# Root Mean Squared Error (RMSE) on test data =  0.07470797713961008

evaluator = RegressionEvaluator(labelCol="medv",\
                                predictionCol="prediction", metricName="r2")
print("R Squared (R2) on test data =", evaluator.evaluate(predictions))

Root Mean Squared Error (RMSE) on test data =  4.671806485171292
R Squared (R2) on test data = 0.7931522547835961
